In [5]:
import tensorflow as tf
from __future__ import print_function

a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)
adder_node = a + b  # + provides a shortcut for tf.add(a, b)
add_and_triple = adder_node * 3

In [7]:
sess = tf.Session()
print(sess.run(add_and_triple, {a: [1, 3], b: [2, 4.5]}))

[  9.   22.5]


In [9]:
W = tf.Variable([0.3], dtype = tf.float32)
b = tf.Variable([-0.3], dtype = tf.float32)
x = tf.placeholder(tf.float32)
linear_model = W*x + b

init = tf.global_variables_initializer()
sess.run(init)

In [10]:
print(sess.run(linear_model, {x: [1, 2, 3, 4]}))


[ 0.          0.30000001  0.60000002  0.90000004]


In [11]:
y = tf.placeholder(tf.float32)
squared_deltas = tf.square(linear_model - y)
loss = tf.reduce_sum(squared_deltas)
print(sess.run(loss, {x: [1, 2, 3, 4], y: [0, -1, -2, -3]}))

23.66


In [12]:
fixW = tf.assign(W, [-1.])
fixb = tf.assign(b, [1.])
sess.run([fixW, fixb])
print(sess.run(loss, {x: [1, 2, 3, 4], y: [0, -1, -2, -3]}))

0.0


In [20]:
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)
sess.run(init)
print(sess.run([W, b]))

for i in range(1000):
    sess.run(train, {x: [1, 2, 3, 4], y: [0, -1, -2, -3]})
    
print(sess.run([W, b]))

[array([ 0.30000001], dtype=float32), array([-0.30000001], dtype=float32)]
[array([-0.9999969], dtype=float32), array([ 0.99999082], dtype=float32)]


# The complete code

In [22]:
import tensorflow as tf

# model parameters
W = tf.Variable([0.3], dtype = tf.float32)
b = tf.Variable([-0.3], dtype = tf.float32)

# Model input and output
x = tf.placeholder(tf.float32)
linear_model = W*x + b
y = tf.placeholder(tf.float32)

# loss
loss = tf.reduce_sum(tf.square(linear_model - y))

# optimizer
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

# training data
x_train = [1, 2, 3, 4]
y_train = [0, -1, -2, -3]

In [23]:
# training loop
init = tf.global_variables_initializer()
sess = tf.Session()

# reset variable values
sess.run(init)

for i in range(1000):
    sess.run(train, {x: x_train, y: y_train})

# evaluate training accuracy
curr_W, curr_b, curr_loss = sess.run([W, b, loss], {x: x_train, y: y_train})
print("W: %s b: %s loss: %s" % (curr_W, curr_b, curr_loss))


W: [-0.9999969] b: [ 0.99999082] loss: 5.69997e-11


# Working with Estimator 

In [25]:
import numpy as np
import tensorflow as tf

# declare list of features
feature_columns = [tf.feature_column.numeric_column("x", shape = [1])]

# provide an estimator for the linear regression
estimator = tf.estimator.LinearRegressor(feature_columns = feature_columns)

# define two data sets: training and evaluation
x_train = np.array([1., 2., 3., 4.])
y_train = np.array([0., -1., -2., -3.])
x_eval = np.array([2., 5., 8., 1.])
y_eval = np.array([-1.01, -4.1, -7, 0.])

# set up number of batches and the size of each batch
input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_train}, y_train, batch_size = 4, num_epochs = None, shuffle = True)
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_train}, y_train, batch_size = 4, num_epochs = 1000, shuffle = False)
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_eval}, y_eval, batch_size = 4, num_epochs = 1000, shuffle = False)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\ro1v0362\\AppData\\Local\\Temp\\tmpo1134uoo', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000000005BC2278>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [26]:
# invoke the 1000 training steps by invoking the method and passing the training set
estimator.train(input_fn = input_fn, steps = 1000)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\ro1v0362\AppData\Local\Temp\tmpo1134uoo\model.ckpt.
INFO:tensorflow:loss = 5.0, step = 1
INFO:tensorflow:global_step/sec: 263.852
INFO:tensorflow:loss = 0.259836, step = 101 (0.380 sec)
INFO:tensorflow:global_step/sec: 261.78
INFO:tensorflow:loss = 0.0334541, step = 201 (0.379 sec)
INFO:tensorflow:global_step/sec: 280.899
INFO:tensorflow:loss = 0.00229999, step = 301 (0.358 sec)
INFO:tensorflow:global_step/sec: 330.033
INFO:tensorflow:loss = 0.000432747, step = 401 (0.304 sec)
INFO:tensorflow:global_step/sec: 369.003
INFO:tensorflow:loss = 3.26236e-05, step = 501 (0.271 sec)
INFO:tensorflow:global_step/sec: 280.899
INFO:tensorflow:loss = 1.20888e-05, step = 601 (0.355 sec)
INFO:tensorflow:global_step/sec: 354.61
INFO:tensorflow:loss = 6.9366e-07, step = 701 (0.282 sec)
INFO:tensorflow:global_step/sec: 317.461
INFO:tensorflow:loss = 2.90783e-07, step = 801 (0.314 sec)
INFO:tensorflow:globa

In [27]:
# evaluate how well the model did
train_metrics = estimator.evaluate(input_fn = train_input_fn)
eval_metrics = estimator.evaluate(input_fn = eval_input_fn)
print("train metrics: %r" % train_metrics)
print("eval metrics: %r" % eval_metrics)

INFO:tensorflow:Starting evaluation at 2017-11-09-14:56:07
INFO:tensorflow:Restoring parameters from C:\Users\ro1v0362\AppData\Local\Temp\tmpo1134uoo\model.ckpt-1000
INFO:tensorflow:Finished evaluation at 2017-11-09-14:56:08
INFO:tensorflow:Saving dict for global step 1000: average_loss = 6.69805e-10, global_step = 1000, loss = 2.67922e-09
INFO:tensorflow:Starting evaluation at 2017-11-09-14:56:09
INFO:tensorflow:Restoring parameters from C:\Users\ro1v0362\AppData\Local\Temp\tmpo1134uoo\model.ckpt-1000
INFO:tensorflow:Finished evaluation at 2017-11-09-14:56:11
INFO:tensorflow:Saving dict for global step 1000: average_loss = 0.00252724, global_step = 1000, loss = 0.0101089
train metrics: {'average_loss': 6.6980499e-10, 'loss': 2.67922e-09, 'global_step': 1000}
eval metrics: {'average_loss': 0.0025272358, 'loss': 0.010108943, 'global_step': 1000}


# Estimator with custom model

In [28]:
import numpy as np
import tensorflow as tf

# declare the model
def model_fn(features, labels, mode):
    # build the linear model
    W = tf.get_variable("W", [1], dtype = tf.float64)
    b = tf.get_variable("b", [1], dtype = tf.float64)
    y = W * features['x'] + b
    
    # loss sub-graph
    loss = tf.reduce_sum(tf.square(y - labels))
    
    # trainig sub-graph
    global_step = tf.train.get_global_step()
    optimizer = tf.train.GradientDescentOptimizer(0.01)
    train = tf.group(
        optimizer.minimize(loss), 
        tf.assign_add(global_step, 1)
    )
    
    # connect estimator with appropriate functionality
    return tf.estimator.EstimatorSpec(
        mode=mode,
        predictions=y,
        loss=loss,
        train_op=train
    )

estimator = tf.estimator.Estimator(model_fn=model_fn)
# define data sets
x_train = np.array([1., 2., 3., 4.])
y_train = np.array([0., -1., -2., -3.])
x_eval = np.array([2., 5., 8., 1.])
y_eval = np.array([-1.01, -4.1, -7., 0.])
input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_train}, y_train, batch_size=4, num_epochs=None, shuffle=True)
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_train}, y_train, batch_size=4, num_epochs=1000, shuffle=False)
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_eval}, y_eval, batch_size=4, num_epochs=1000, shuffle=False)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\ro1v0362\\AppData\\Local\\Temp\\tmpenfl_y4o', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000000001C3FB550>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [29]:
# train
estimator.train(input_fn=input_fn, steps=1000)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\ro1v0362\AppData\Local\Temp\tmpenfl_y4o\model.ckpt.
INFO:tensorflow:loss = 5.16443864802, step = 1
INFO:tensorflow:global_step/sec: 323.625
INFO:tensorflow:loss = 0.35670102562, step = 101 (0.309 sec)
INFO:tensorflow:global_step/sec: 323.624
INFO:tensorflow:loss = 0.0712856835038, step = 201 (0.310 sec)
INFO:tensorflow:global_step/sec: 476.191
INFO:tensorflow:loss = 0.00453609964355, step = 301 (0.209 sec)
INFO:tensorflow:global_step/sec: 581.395
INFO:tensorflow:loss = 0.000297850604731, step = 401 (0.172 sec)
INFO:tensorflow:global_step/sec: 346.021
INFO:tensorflow:loss = 1.28573547869e-05, step = 501 (0.289 sec)
INFO:tensorflow:global_step/sec: 337.838
INFO:tensorflow:loss = 3.20050054432e-06, step = 601 (0.294 sec)
INFO:tensorflow:global_step/sec: 393.7
INFO:tensorflow:loss = 2.03433312353e-07, step = 701 (0.254 sec)
INFO:tensorflow:global_step/sec: 398.406
INFO:tensorflow:loss = 2.800

In [30]:
# evaluate the model
train_metrics = estimator.evaluate(input_fn=train_input_fn)
eval_metrics = estimator.evaluate(input_fn=eval_input_fn)
print("train metrics: %r"% train_metrics)
print("eval metrics: %r"% eval_metrics)

INFO:tensorflow:Starting evaluation at 2017-11-09-15:07:44
INFO:tensorflow:Restoring parameters from C:\Users\ro1v0362\AppData\Local\Temp\tmpenfl_y4o\model.ckpt-1000
INFO:tensorflow:Finished evaluation at 2017-11-09-15:07:45
INFO:tensorflow:Saving dict for global step 1000: global_step = 1000, loss = 1.39765e-10
INFO:tensorflow:Starting evaluation at 2017-11-09-15:07:46
INFO:tensorflow:Restoring parameters from C:\Users\ro1v0362\AppData\Local\Temp\tmpenfl_y4o\model.ckpt-1000
INFO:tensorflow:Finished evaluation at 2017-11-09-15:07:47
INFO:tensorflow:Saving dict for global step 1000: global_step = 1000, loss = 0.0101023
train metrics: {'loss': 1.3976525e-10, 'global_step': 1000}
eval metrics: {'loss': 0.010102277, 'global_step': 1000}
